In [12]:
import requests
import json

In [24]:
class Annapurna_Scraper:
    def __init__(self,page_no,search_keyword = 'चुनाव',save_to_file=True):
        self.search_keyword = search_keyword
        self.page_no = page_no
        self.save_to_file = save_to_file
        self.data = []
        
    def fetch_data(self,search_key,page):
        url = f"https://bg.annapurnapost.com/api/search?title={search_key}&page={page}"
        header = {
            'authority': 'bg.annapurnapost.com',
            'content-type': 'application/json',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
            'cache-control': 'no-cache',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',

        }
        return requests.get(url,headers=header)
    def Scrape(self):
        try:
            temp = []
            s_index = 0
            try:
                with open('scrape.json','r') as f:
                    data_from_file = json.loads(f.read())
                file_flag = True
            except:
                file_flag = False
            for page in range(self.page_no):
                if file_flag == True:  
                    try:  
                        for file in data_from_file:
                            try:
                                if (len(file) == 1 and file['page'] == page+1):
                                    f_index = data_from_file.index(file)
                                    print(f'page {page+1} already found in the disk.....')
                                    temp+=data_from_file[s_index:f_index]
                                    s_index = f_index
                                    page_flag = True
                                    break
                                else:
                                    page_flag = False
                            except:
                                continue
                        if page_flag == True:
                            continue
                    except Exception as e:
                        print(e)
                print(f'Downloading page {page+1}....')
                response = self.fetch_data(self.search_keyword,page+1)
                if response.status_code != 200 :
                    raise Exception('Connection error !!!')
                else:
                    data = response.json()['data']['items']
                    data.append({'page' : page+1}) 
                    self.data = self.data + data
            if self.save_to_file == True:
                if len(self.data) == 0:
                    return None
                self.data+=temp
                with open('scrape.json','a',encoding='utf8') as f:
                    json.dump(self.data,f)
            else:    
                return self.data
        except Exception as e:
            print(e)


scrape = Annapurna_Scraper(search_keyword='कांग्रेस',page_no=2,save_to_file=False)
data = scrape.Scrape() 

In [ ]:
''' Method 1. List of Dictionaries '''
def voting_list(state,ward_no):
    payload = {
        'state':f'{state}',
        'district':'26',
        'vdc_mun':'5278',
        'ward':f'{ward_no}',
        'reg_centre':''
            }
    url = 'https://voterlist.election.gov.np/bbvrs/view_ward_1.php'
    response = requests.post(url,data=payload)
    soup = BeautifulSoup(response.text,"html.parser") 
    div = soup.find('div',class_='div_bbvrs_data')
    table = div.find("table")
    head_table_data = ['voter_id','voter_name','age','gender','wife/husband_name','father/mother_name']
    body_table_data = table.tbody.find_all("tr")  
    list_data = []
    for count in range(len(body_table_data)):
        dict_data = {}
        # for data in body_table_data[i].find_all("td"):
        for i,th in enumerate(head_table_data):
            dict_data[th[i]] = body_table_data[count].find_all("td")[i+1].get_text()   
        list_data.append(dict_data)
    string_list_data = str(list_data)
    numbers_map = {'0': '०','1': '१','2': '२','3': '३','4': '४','5': '५','6': '६','7': '७','8': '८','9': '९'}
    for eng_num,unicode in numbers_map.items():
        string_list_data = string_list_data.replace(eng_num,unicode)
    bytestring = string_list_data.encode("utf-8")
    json_str = json.dumps(string_list_data,ensure_ascii=False).encode('utf8')
    with open(f'voting_list_wards{ward_no}.json','wb',encoding='utf8') as file:
        file.write(bytestring)

In [27]:
import json,os,requests

'''saves file in a folder on the basis of page.'''
class Annapurna_Scraper:
    '''
        Annapurna posts scraper: 
            It scrapes the posts on the given search_keyword parameter page by page
            Scrape up to arround 20 pages at a time; 
        
        How to use:
        -- Create an instance of Annapurna_Scraper class
        -- While creating instance provide three parameters shown below
        -- Call Scrape() method with the instance
        -- This method returns data but if save_to_file=True then it also saves to 'scrape.json' 
            file on the current directory 
        
        >>>> s = Annapurna_Scraper(,page_no=5,search_keyword = 'चुनाव',save_to_file=True)
        >>>> data = s.Scrape()
    '''
    def __init__(self,page_no,search_keyword = 'चुनाव',save_to_file=True):
        self.search_keyword = search_keyword
        self.page_no = page_no
        self.save_to_file = save_to_file
        self.response_data = []
        
    def request_data(self,page):
        url = f"https://bg.annapurnapost.com/api/search?title={self.search_keyword}&page={page}"
        header = {
            'authority': 'bg.annapurnapost.com',
            'content-type': 'application/json',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
            'cache-control': 'no-cache',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',

        }
        return requests.get(url,headers=header)
    
    def fetch_data(self,page):
        print(f'Downloading page {page}....')
        response = self.request_data(page)
        if response.status_code != 200 :
            raise Exception('Connection error !!!')
        else:
            return response.json()['data']['items']
            
    def save_data(self,data,page):
        print('saving...')
        if not os.path.exists(f'annapurna-scrape/{self.search_keyword}'):
            os.mkdir(f"annapurna-scrape/{self.search_keyword}")
        with open(f'annapurna-scrape/{self.search_keyword}/{page}.json','w',encoding='utf8') as f:
            json.dump(data,f,ensure_ascii=False)
        return data

    def Scrape(self): 
        for page in range(1,self.page_no+1):
            try:
                with open(f'annapurna-scrape/{self.search_keyword}/{page}.json','r') as f:
                    data_from_file = json.loads(f.read())
                file_flag = True
            except:
                file_flag = False
            
            if file_flag == True:
                if len(data_from_file) != 0 :
                    print(f'page {page} already found in the disk.....')
                    continue

            else:
                data = self.fetch_data(page)
                self.response_data.append(self.save_data(data,page))
                    
        if self.save_to_file == True:
            return None
        else:
            return self.response_data

scrape = Annapurna_Scraper(search_keyword='कांग्रेस',page_no=6,save_to_file=True)
data = scrape.Scrape() 


saving...
saving...
saving...
saving...
saving...
saving...
